# Packages & Data Loading

In [122]:
import pandas as pd
from statsmodels.tsa.stattools import adfuller
import matplotlib.pyplot as plt
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

In [123]:
# Load datasets
unemployment_df = pd.read_csv("D:\\OneDrive (Personal)\\OneDrive\\~ TMU 2023\\CIND 860 - Adv. Data Analytics Project\\05 - Lit. Review, Data Description (25%)\\Datasets - Raw\\Unemployment Rate\\Unemployment Dataset_Selected Variables.csv")
wage_df = pd.read_csv("D:\\OneDrive (Personal)\\OneDrive\\~ TMU 2023\\CIND 860 - Adv. Data Analytics Project\\05 - Lit. Review, Data Description (25%)\\Datasets - Raw\\Wage Rate\\Wage Dataset_Selected Variables.csv")

In [124]:
# Check and drop unnecessary columns
columns_to_drop = ['DGUID', 'UOM', 'UOM_ID', 'SCALAR_FACTOR', 'SCALAR_ID', 'VECTOR', 'COORDINATE', 'STATUS', 'SYMBOL', 'TERMINATED', 'DECIMALS']
columns_to_drop_unemployment = [col for col in columns_to_drop if col in unemployment_df.columns]
columns_to_drop_wage = [col for col in columns_to_drop if col in wage_df.columns]

data_employment = unemployment_df.drop(columns=columns_to_drop_unemployment)
data_wage = wage_df.drop(columns=columns_to_drop_wage)

# Ensure column names are normalized (lowercase and replace spaces with underscores)
data_employment.columns = [col.lower().replace(' ', '_') for col in data_employment.columns]
data_wage.columns = [col.lower().replace(' ', '_') for col in data_wage.columns]

# Rename the column from 'north_american_industry_classification_system_(naics)' to 'industry'
data_employment.rename(columns={'north_american_industry_classification_system_(naics)': 'industry'}, inplace=True)
data_wage.rename(columns={'north_american_industry_classification_system_(naics)': 'industry'}, inplace=True)

# Rename the column from 'labour_force_characteristics' to 'labour_force'
data_employment.rename(columns={'labour_force_characteristics': 'labour_force'}, inplace=True)
data_wage.rename(columns={'labour_force_characteristics': 'labour_force'}, inplace=True)

In [125]:
print(data_employment.head())

   ref_date                        geo       labour_force  \
0      1997  Newfoundland and Labrador  Unemployment rate   
1      1998  Newfoundland and Labrador  Unemployment rate   
2      1999  Newfoundland and Labrador  Unemployment rate   
3      2000  Newfoundland and Labrador  Unemployment rate   
4      2001  Newfoundland and Labrador  Unemployment rate   

                                 industry    sex       age_group  value  
0  Agriculture [111-112, 1100, 1151-1152]  Males  15 to 24 years    NaN  
1  Agriculture [111-112, 1100, 1151-1152]  Males  15 to 24 years    NaN  
2  Agriculture [111-112, 1100, 1151-1152]  Males  15 to 24 years    NaN  
3  Agriculture [111-112, 1100, 1151-1152]  Males  15 to 24 years    NaN  
4  Agriculture [111-112, 1100, 1151-1152]  Males  15 to 24 years    NaN  


In [126]:
print(data_wage.head())

   ref_date                        geo                     wages  \
0      1997  Newfoundland and Labrador  Average hourly wage rate   
1      1998  Newfoundland and Labrador  Average hourly wage rate   
2      1999  Newfoundland and Labrador  Average hourly wage rate   
3      2000  Newfoundland and Labrador  Average hourly wage rate   
4      2001  Newfoundland and Labrador  Average hourly wage rate   

          type_of_work                                industry    sex  \
0  Full-time employees  Agriculture [111-112, 1100, 1151-1152]  Males   
1  Full-time employees  Agriculture [111-112, 1100, 1151-1152]  Males   
2  Full-time employees  Agriculture [111-112, 1100, 1151-1152]  Males   
3  Full-time employees  Agriculture [111-112, 1100, 1151-1152]  Males   
4  Full-time employees  Agriculture [111-112, 1100, 1151-1152]  Males   

        age_group  value  
0  15 to 24 years    NaN  
1  15 to 24 years    NaN  
2  15 to 24 years    NaN  
3  15 to 24 years    NaN  
4  15 to 24 years

## Merge the datasets

In [127]:
print(data_employment.columns)
print(data_wage.columns)


Index(['ref_date', 'geo', 'labour_force', 'industry', 'sex', 'age_group',
       'value'],
      dtype='object')
Index(['ref_date', 'geo', 'wages', 'type_of_work', 'industry', 'sex',
       'age_group', 'value'],
      dtype='object')


In [128]:
# Merge datasets on common columns
merged_df = pd.merge(data_employment, data_wage, 
                     on=["ref_date", "geo", "industry", "sex", "age_group"],
                     how="outer",
                     suffixes=('_unemployment', '_wage'))


In [129]:
from ydata_profiling import ProfileReport


# Generate the report
profile = ProfileReport(merged_df,title="EDA Report")

# Save the report to .html
profile.to_file("eda_report.html")

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

# Initial Data Inspection

In [130]:
# Dataset Overview
print("\n", merged_df.head(3),"\n") # preview a sample

print("Number of obs and features:", merged_df.shape,"\n")  # number of observations and features

print("Data types:", merged_df.dtypes,"\n") # data types

print("Number of duplicated values:", merged_df[merged_df.duplicated()],"\n") # check duplicated values

print("Number of missing values per feature:", merged_df.isna().sum(),"\n") # missing values per feature

print("Number of missing values:", merged_df.isna().sum().sum(),"\n") # number of missing cells
print("Percentage of missing values:", round(merged_df.isna().sum().sum() / merged_df.size * 100, 1),"\n") # percentage of missing cells



    ref_date      geo       labour_force                              industry  \
0      1997  Alberta  Unemployment rate  Accommodation and food services [72]   
1      1997  Alberta  Unemployment rate  Accommodation and food services [72]   
2      1997  Alberta  Unemployment rate  Accommodation and food services [72]   

       sex       age_group  value_unemployment                     wages  \
0  Females  15 to 24 years                 8.2  Average hourly wage rate   
1  Females  15 to 24 years                 8.2  Average hourly wage rate   
2  Females  25 to 54 years                 5.0  Average hourly wage rate   

          type_of_work  value_wage  
0  Full-time employees         7.3  
1  Part-time employees         6.1  
2  Full-time employees         8.7   

Number of obs and features: (55080, 10) 

Data types: ref_date                int64
geo                    object
labour_force           object
industry               object
sex                    object
age_group     

In [131]:
# Value counts for all columns in merged_df
for column in merged_df.columns:
    print(f"Value counts for column '{column}':")
    print(merged_df[column].value_counts())
    print("\n" + "-"*50 + "\n")  # Separator for readability


Value counts for column 'ref_date':
ref_date
1997    2040
2011    2040
2022    2040
2021    2040
2020    2040
2019    2040
2018    2040
2017    2040
2016    2040
2015    2040
2014    2040
2013    2040
2012    2040
2010    2040
1998    2040
2009    2040
2008    2040
2007    2040
2006    2040
2005    2040
2004    2040
2003    2040
2002    2040
2001    2040
2000    2040
1999    2040
2023    2040
Name: count, dtype: int64

--------------------------------------------------

Value counts for column 'geo':
geo
Alberta                      5508
British Columbia             5508
Manitoba                     5508
New Brunswick                5508
Newfoundland and Labrador    5508
Nova Scotia                  5508
Ontario                      5508
Prince Edward Island         5508
Quebec                       5508
Saskatchewan                 5508
Name: count, dtype: int64

--------------------------------------------------

Value counts for column 'labour_force':
labour_force
Unemployment rate 

# Missing Values 

## Inspect Missing Values

In [132]:
# Filter rows based on the conditions:
# 1. There is data in 'labour force' and a missing value in 'value_unemployment'.
# 2. There is data in 'wages' and a missing value in 'type_of_work' AND/OR 'value_wage'.
data_missing = merged_df[
    ((merged_df['labour_force'].notna()) & (merged_df['value_unemployment'].isna())) |
    ((merged_df['wages'].notna()) & (merged_df[['type_of_work', 'value_wage']].isna().any(axis=1)))
]

# Total number of rows in the original dataset
total_rows = merged_df.shape[0]

# Count the number of rows with missing data
missing_rows = data_missing.shape[0]

# Calculate the percentage of rows with missing data
missing_percentage = (missing_rows / total_rows) * 100

# Output the result
print(f"Percentage of rows with missing data: {missing_percentage:.2f}%")


Percentage of rows with missing data: 73.21%


In [133]:
#Value counts for all columns
for column in data_missing.columns:
    print(f"Value counts for column '{column}':")
    print(data_missing[column].value_counts())
    print("\n" + "-"*50 + "\n")  # Separator for readability


Value counts for column 'ref_date':
ref_date
2022    1567
2001    1551
2000    1549
2023    1541
2018    1539
2005    1536
2006    1526
2007    1523
2008    1521
2013    1510
1998    1507
2003    1500
2017    1500
2019    1499
1997    1498
2002    1498
2014    1494
1999    1490
2004    1488
2011    1482
2016    1477
2012    1470
2015    1470
2021    1443
2010    1441
2009    1437
2020    1265
Name: count, dtype: int64

--------------------------------------------------

Value counts for column 'geo':
geo
Prince Edward Island         4845
Newfoundland and Labrador    4677
Saskatchewan                 4507
New Brunswick                4453
Manitoba                     4310
Alberta                      4213
Nova Scotia                  4176
British Columbia             3831
Quebec                       2952
Ontario                      2358
Name: count, dtype: int64

--------------------------------------------------

Value counts for column 'labour_force':
labour_force
Unemployment rate 

In [134]:
# Percentage value counts for all columns
for column in data_missing.columns:
    total_count = len(data_missing[column])
    print(f"Percentage counts for column '{column}':")
    
    # Calculate the value counts and convert them to percentages
    percentage_counts = (data_missing[column].value_counts() / total_count) * 100
    
    print(percentage_counts)
    print("\n" + "-"*50 + "\n")  # Separator for readability


Percentage counts for column 'ref_date':
ref_date
2022    3.886216
2001    3.846535
2000    3.841575
2023    3.821735
2018    3.816775
2005    3.809335
2006    3.784534
2007    3.777094
2008    3.772134
2013    3.744854
1998    3.737414
2003    3.720054
2017    3.720054
2019    3.717574
1997    3.715093
2002    3.715093
2014    3.705173
1999    3.695253
2004    3.690293
2011    3.675413
2016    3.663013
2012    3.645652
2015    3.645652
2021    3.578692
2010    3.573731
2009    3.563811
2020    3.137245
Name: count, dtype: float64

--------------------------------------------------

Percentage counts for column 'geo':
geo
Prince Edward Island         12.015773
Newfoundland and Labrador    11.599127
Saskatchewan                 11.177521
New Brunswick                11.043599
Manitoba                     10.688954
Alberta                      10.448390
Nova Scotia                  10.356629
British Columbia              9.501017
Quebec                        7.321065
Ontario            

## Missing Values (Linear Interpolation)

In [135]:
# Make a copy of the data
interpolated_data = merged_df.copy()

# Apply linear interpolation on the 'value' column in the new DataFrame
interpolated_data['value_unemployment'] = interpolated_data['value_unemployment'].interpolate(method='linear')
interpolated_data['value_wage'] = interpolated_data['value_wage'].interpolate(method='linear')

# Verify that missing values have been handled in the new DataFrame
print("\nMissing Unemployment Values in Original Data:\n", merged_df['value_unemployment'].isnull().sum())
print("\nMissing Wage Values in Original Data:\n", merged_df['value_wage'].isnull().sum())

print("\nMissing Unemployment Values After Interpolation in New DataFrame:\n", interpolated_data['value_unemployment'].isnull().sum())
print("\nMissing Wage Values After Interpolation in New DataFrame:\n", interpolated_data['value_wage'].isnull().sum())



Missing Unemployment Values in Original Data:
 42744

Missing Wage Values in Original Data:
 23376

Missing Unemployment Values After Interpolation in New DataFrame:
 0

Missing Wage Values After Interpolation in New DataFrame:
 0


In [136]:
# Ensure 'ref_date' is in datetime format and set it as the index
interpolated_data['ref_date'] = pd.to_datetime(interpolated_data['ref_date'], format='%Y')
interpolated_data.set_index('ref_date', inplace=True)

### ADF - Unemployment 

In [137]:
# Perform ADF test on 'value' column to check for stationarity
adf_test = adfuller(interpolated_data['value_unemployment'])
print("\nADF Test Results:")
print(f"ADF Statistic: {adf_test[0]}")
print(f"p-value: {adf_test[1]}")
if adf_test[1] < 0.05:
    print("The data is stationary.")
else:
    print("The data is not stationary. Differencing may be required.")


ADF Test Results:
ADF Statistic: -18.20680945063685
p-value: 2.3982079546259774e-30
The data is stationary.


In [138]:
# Plot ACF and PACF to determine p and q
plt.figure(figsize=(12, 6))
plt.subplot(121)
plot_acf(interpolated_data['value_diff'].dropna() if 'value_diff' in interpolated_data else interpolated_data['value_unemployment'], 
         ax=plt.gca(), lags=40)
plt.title("ACF Plot")
plt.subplot(122)
plot_pacf(interpolated_data['value_diff'].dropna() if 'value_diff' in interpolated_data else interpolated_data['value_unemployment'], 
          ax=plt.gca(), lags=40)
plt.title("PACF Plot")

# Save the figure to a file
plt.savefig("acf_pacf_plot_unemployment.png")

# Inform the user that the plot was saved
print("ACF and PACF plots saved as 'acf_pacf_plot_unemployment.png'.")

ACF and PACF plots saved as 'acf_pacf_plot_unemployment.png'.


### ADF - Wage

In [139]:
# Perform ADF test on 'value' column to check for stationarity
adf_test = adfuller(interpolated_data['value_wage'])
print("\nADF Test Results:")
print(f"ADF Statistic: {adf_test[0]}")
print(f"p-value: {adf_test[1]}")
if adf_test[1] < 0.05:
    print("The data is stationary.")
else:
    print("The data is not stationary. Differencing may be required.")


ADF Test Results:
ADF Statistic: -16.259276826690805
p-value: 3.579486710351117e-29
The data is stationary.


In [140]:
import matplotlib.pyplot as plt
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

# Plot ACF and PACF to determine p and q
plt.figure(figsize=(12, 6))
plt.subplot(121)
plot_acf(interpolated_data['value_diff'].dropna() if 'value_diff' in interpolated_data else interpolated_data['value_wage'], 
         ax=plt.gca(), lags=40)
plt.title("ACF Plot")
plt.subplot(122)
plot_pacf(interpolated_data['value_diff'].dropna() if 'value_diff' in interpolated_data else interpolated_data['value_wage'], 
          ax=plt.gca(), lags=40)
plt.title("PACF Plot")

# Save the figure to a file
plt.savefig("acf_pacf_plot_wage.png")

# Inform the user that the plot was saved
print("ACF and PACF plots saved as 'acf_pacf_plot.png_wage'.")

ACF and PACF plots saved as 'acf_pacf_plot.png_wage'.
